In [39]:
import sys
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm, tqdm_notebook
import datetime
import xgboost
import catboost as cb
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,cross_val_score, train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.externals import joblib

import data_helpers as dh
import data_iter1 as di
import submit_report as rep
import model_utils as mu

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# загрузка данных
events  = pd.read_csv('event_data_train.csv')
submissions = pd.read_csv('submissions_data_train.csv')

# разделение данных для обучения на train и test
_ = dh.split_events_submissions(events, submissions, test_size=0.3)
events_train_orig, events_test_orig, submissions_train_orig, submissions_test_orig = _

# подготовка данных
X_train, y_train = di.get_x_y(events_train_orig, submissions_train_orig)
X_test, y_test = di.get_x_y(events_test_orig, submissions_test_orig)

#загрузка данных для предсказаний
events_pred  = pd.read_csv('events_data_test.csv')
submissions_pred = pd.read_csv('submission_data_test.csv')
X_pred , _ = di.get_x_y(events_pred, submissions_pred)

D:\Test\ods_course\data_helpers.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['date'] = pd.to_datetime(data.timestamp, unit='s')
D:\Test\ods_course\data_helpers.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['day'] = data.date.dt.date


In [3]:
#Подготовка данных для тренировки модели
X = X_train.append(X_test)
y = y_train.append(y_test)
user_min_time = events.groupby('user_id', as_index=False).agg({'timestamp':'min'})
X['user_id'] = X.index
X = X.merge(user_min_time[['user_id','timestamp']], how='outer')
X['date'] = pd.to_datetime(X['timestamp'], unit = 's')
X['day'] = X['date'].dt.date
X['weekday'] = X['date'].dt.dayofweek
#X = X.drop('user_id', axis=1)
X = X.drop('timestamp', axis=1)
X['year'] = X['date'].dt.year
X['month'] = X['date'].dt.month
X['hour'] = X['date'].dt.hour
X = X.drop('day', axis=1)
X = X.drop('date', axis=1)
X.shape

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 'user_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  


(19234, 11)

In [4]:
#подготовка данных для предсказаний
user_min_time_pred = events_pred.groupby('user_id', as_index=False).agg({'timestamp':'min'})
X_pred['user_id'] = X_pred.index
X_pred = X_pred.merge(user_min_time_pred[['user_id','timestamp']], how='outer')
X_pred['date'] = pd.to_datetime(X_pred['timestamp'], unit = 's')
X_pred['day'] = X_pred['date'].dt.date
X_pred['weekday'] = X_pred['date'].dt.dayofweek
X_pred = X_pred.drop('timestamp', axis=1)
X_pred['year'] = X_pred['date'].dt.year
X_pred['month'] = X_pred['date'].dt.month
X_pred['hour'] = X_pred['date'].dt.hour
X_pred = X_pred.drop('day', axis=1)
X_pred = X_pred.drop('date', axis=1)
X_pred.shape

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 'user_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  after removing the cwd from sys.path.


(6184, 11)

In [5]:
#данные для OneHotEncoder
year = [2015, 2016, 2017, 2018, 2019]
month = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
weekday = [0, 1, 2, 3, 4, 5, 6]
hour = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

In [6]:
for i in year:
    X['year='+str(i)] = (X['year']==i).astype('float')
    X_pred['year='+str(i)] = (X_pred['year']==i).astype('float')
for i in month:
    X['month='+str(i)] = (X['month']==i).astype('float')
    X_pred['month='+str(i)] = (X_pred['month']==i).astype('float')
for i in weekday:
    X['weekday='+str(i)] = (X['weekday']==i).astype('float')
    X_pred['weekday='+str(i)] = (X_pred['weekday']==i).astype('float')
for i in hour:
    X['hour='+str(i)] = (X['hour']==i).astype('float')
    X_pred['hour='+str(i)] = (X_pred['hour']==i).astype('float')
X.head()

,correct,wrong,discovered,passed,started_attempt,viewed,user_id,weekday,year,month,...,hour=14,hour=15,hour=16,hour=17,hour=18,hour=19,hour=20,hour=21,hour=22,hour=23
0,0.0,0.0,1,0,0,1,1,4,2016,9,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,9,9,2,9,2,2,2017,12,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,4.0,15,15,4,20,3,0,2015,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,2.0,1,1,0,1,5,4,2016,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1,0,1,1,11,5,2018,5,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
X = X.drop('year',axis=1)
X = X.drop('month',axis=1)
X = X.drop('weekday',axis=1)
X = X.drop('hour',axis=1)
X_pred = X_pred.drop('year',axis=1)
X_pred = X_pred.drop('month',axis=1)
X_pred = X_pred.drop('weekday',axis=1)
X_pred = X_pred.drop('hour',axis=1)
X = X.set_index('user_id')
X_pred = X_pred.set_index('user_id')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=2, 
                            min_samples_leaf=10, min_samples_split=10, 
                            class_weight='balanced')
rf.fit(X_train, y_train)
pred_proba = rf.predict_proba(X_test)
roc_score = roc_auc_score(y_test, pred_proba[:, 1])
print('roc на test', roc_score)
# должны получить roc 0.92  +- 0.02

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=10,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=2, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

roc на test 0.9306276054542599


In [9]:
# важность фич
fimp = mu.get_feature_importances_df(rf.feature_importances_, X_train.columns)
fimp.head(15)

,weight
correct,0.295014
passed,0.168456
discovered,0.136585
viewed,0.126764
wrong,0.108897
started_attempt,0.096968
month=6,0.010910
year=2015,0.003888
year=2016,0.002969
year=2018,0.002899


In [10]:
# значение к метрике на кроссвалидации коррелирует к метрике на степике
rfcv = RandomForestClassifier(**rf.get_params())
cv_scores = cross_val_score(rfcv, X_train, y_train, scoring='roc_auc', cv=10, n_jobs=-1)
mean_cv_scores = np.mean(cv_scores)
print ('mean score', mean_cv_scores)

mean score 0.9250867351816716


In [36]:
SUBMIT_NUM = 9
pred_proba = rf.predict_proba(X_pred)[:, 1]
rep_df = rep.create_report(X_pred.index, pred_proba)
assert rep_df.user_id.nunique() == X_pred.index.nunique()
print ('Прогноз сохранен в файл ', rep.save_report(rep_df, SUBMIT_NUM))

print ('Распределение "вероятностей" модели')
pd.cut(pred_proba, 10).value_counts()

Прогноз сохранен в файл  predict_2019-05-22_submit_9.csv
Распределение "вероятностей" модели


(-0.000228, 0.1]    3221
(0.1, 0.2]           223
(0.2, 0.3]           369
(0.3, 0.399]         572
(0.399, 0.499]       348
(0.499, 0.599]       234
(0.599, 0.698]       292
(0.698, 0.798]       334
(0.798, 0.897]       130
(0.897, 0.997]       461
dtype: int64

In [11]:
rf = RandomForestClassifier(random_state=0)
parametrs_rf = {'criterion': ['gini', 'entropy'],
            'max_features' : [9, 10],
            'n_estimators': [15, 20, 50, 100],
            'max_depth': [4, 5, 6],
            'min_samples_leaf': [1, 2, 5, 10],
            'min_samples_split': [10, 15, 20]}
grid_search_cv_clf = GridSearchCV(estimator=rf, param_grid=parametrs_rf, cv=5, n_jobs=-1, verbose=1, scoring='roc_auc')
grid_search_cv_clf.fit(X_train, y_train)

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 2880 out of 2880 | elapsed:  6.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'max_features': [9, 10], 'n_estimators': [15, 20, 50, 100], 'max_depth': [4, 5, 6], 'min_samples_leaf': [1, 2, 5, 10], 'min_samples_split': [10, 15, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [14]:
best_rf = grid_search_cv_clf.best_estimator_
pred_proba_rf = best_rf.predict_proba(X_test)
roc_score = roc_auc_score(y_test, pred_proba_rf[:, 1])
print('roc на test', roc_score)

roc на test 0.9282151803519165


In [15]:
xgb_model = xgboost.XGBClassifier()
test_params = {
  "n_estimators" : [90, 100, 110],
 "learning_rate"    : [0.02, 0.04, 0.05],
 "max_depth"        : [3, 4, 5],
 "min_child_weight" : [ 1, 2, 4]
}
model = GridSearchCV(estimator = xgb_model,param_grid = test_params, n_jobs=-1, verbose=10, cv=5)

In [16]:
model.fit(X_train,y_train)
best_xgb = model.best_estimator_
pred_proba_xgb = best_xgb.predict_proba(X_test)
roc_score = roc_auc_score(y_test, pred_proba_xgb[:, 1])
print('roc на test', roc_score)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   43.0s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   55.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  4

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [90, 100, 110], 'learning_rate': [0.02, 0.04, 0.05], 'max_depth': [3, 4, 5], 'min_child_weight': [1, 2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

roc на test 0.9316927514768635


In [26]:
pred_proba_mean = pred_proba_rf[:, 1]*0.1 + pred_proba_xgb[:, 1]*0.9
roc_score = roc_auc_score(y_test, pred_proba_mean)
print('roc на test', roc_score)

roc на test 0.9318566651591451


In [36]:
y_train_int = y_train.map(lambda x: 1 if (x==True) else 0)
y_test_int = y_test.map(lambda x: 1 if (x==True) else 0)

In [47]:
"""params = {'depth': [4, 7, 10],
          'learning_rate' : [0.01, 0.02, 0.03, 0.035658, 0.04, 0.05],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
"""
params = {'learning_rate' : [0.01, 0.02, 0.03, 0.035658, 0.04, 0.05]}
cb_clf = cb.CatBoostClassifier()
cb_model = GridSearchCV(cb_clf, params, scoring="roc_auc", cv = 5)
cb_model.fit(X_train, y_train_int)

"params = {'depth': [4, 7, 10],\n          'learning_rate' : [0.01, 0.02, 0.03, 0.035658, 0.04, 0.05],\n         'l2_leaf_reg': [1,4,9],\n         'iterations': [300]}\n"

0:	learn: 0.6828094	total: 55.4ms	remaining: 55.3s
1:	learn: 0.6692514	total: 128ms	remaining: 1m 3s
2:	learn: 0.6585997	total: 184ms	remaining: 1m
3:	learn: 0.6484599	total: 241ms	remaining: 1m
4:	learn: 0.6360522	total: 302ms	remaining: 1m
5:	learn: 0.6263738	total: 363ms	remaining: 1m
6:	learn: 0.6171194	total: 422ms	remaining: 59.8s
7:	learn: 0.6088103	total: 476ms	remaining: 59.1s
8:	learn: 0.6006193	total: 545ms	remaining: 60s
9:	learn: 0.5897611	total: 614ms	remaining: 1m
10:	learn: 0.5796576	total: 666ms	remaining: 59.9s
11:	learn: 0.5692044	total: 714ms	remaining: 58.8s
12:	learn: 0.5598088	total: 763ms	remaining: 57.9s
13:	learn: 0.5529982	total: 812ms	remaining: 57.2s
14:	learn: 0.5451740	total: 859ms	remaining: 56.4s
15:	learn: 0.5381729	total: 929ms	remaining: 57.1s
16:	learn: 0.5305530	total: 999ms	remaining: 57.8s
17:	learn: 0.5243022	total: 1.07s	remaining: 58.5s
18:	learn: 0.5175105	total: 1.13s	remaining: 58.4s
19:	learn: 0.5128155	total: 1.18s	remaining: 58s
20:	lear

162:	learn: 0.2924215	total: 8.37s	remaining: 43s
163:	learn: 0.2919580	total: 8.41s	remaining: 42.9s
164:	learn: 0.2916475	total: 8.46s	remaining: 42.8s
165:	learn: 0.2914992	total: 8.51s	remaining: 42.7s
166:	learn: 0.2910931	total: 8.55s	remaining: 42.7s
167:	learn: 0.2909057	total: 8.6s	remaining: 42.6s
168:	learn: 0.2907707	total: 8.64s	remaining: 42.5s
169:	learn: 0.2903654	total: 8.68s	remaining: 42.4s
170:	learn: 0.2903032	total: 8.73s	remaining: 42.3s
171:	learn: 0.2898091	total: 8.77s	remaining: 42.2s
172:	learn: 0.2897274	total: 8.81s	remaining: 42.1s
173:	learn: 0.2896411	total: 8.85s	remaining: 42s
174:	learn: 0.2895378	total: 8.9s	remaining: 41.9s
175:	learn: 0.2892454	total: 8.94s	remaining: 41.9s
176:	learn: 0.2889498	total: 8.99s	remaining: 41.8s
177:	learn: 0.2888493	total: 9.03s	remaining: 41.7s
178:	learn: 0.2886914	total: 9.08s	remaining: 41.6s
179:	learn: 0.2886059	total: 9.13s	remaining: 41.6s
180:	learn: 0.2881651	total: 9.19s	remaining: 41.6s
181:	learn: 0.2880

322:	learn: 0.2732629	total: 17.1s	remaining: 35.9s
323:	learn: 0.2731370	total: 17.2s	remaining: 35.9s
324:	learn: 0.2729998	total: 17.3s	remaining: 35.8s
325:	learn: 0.2729722	total: 17.3s	remaining: 35.8s
326:	learn: 0.2729289	total: 17.4s	remaining: 35.7s
327:	learn: 0.2729059	total: 17.4s	remaining: 35.7s
328:	learn: 0.2728279	total: 17.5s	remaining: 35.7s
329:	learn: 0.2727819	total: 17.6s	remaining: 35.7s
330:	learn: 0.2727581	total: 17.6s	remaining: 35.6s
331:	learn: 0.2727496	total: 17.7s	remaining: 35.5s
332:	learn: 0.2726925	total: 17.7s	remaining: 35.5s
333:	learn: 0.2726540	total: 17.7s	remaining: 35.4s
334:	learn: 0.2726393	total: 17.8s	remaining: 35.3s
335:	learn: 0.2726291	total: 17.9s	remaining: 35.3s
336:	learn: 0.2725990	total: 17.9s	remaining: 35.3s
337:	learn: 0.2725873	total: 18s	remaining: 35.3s
338:	learn: 0.2725528	total: 18.1s	remaining: 35.2s
339:	learn: 0.2725052	total: 18.1s	remaining: 35.1s
340:	learn: 0.2724989	total: 18.1s	remaining: 35.1s
341:	learn: 0.

481:	learn: 0.2678381	total: 25.3s	remaining: 27.2s
482:	learn: 0.2677995	total: 25.3s	remaining: 27.1s
483:	learn: 0.2677588	total: 25.4s	remaining: 27s
484:	learn: 0.2677275	total: 25.4s	remaining: 27s
485:	learn: 0.2677074	total: 25.5s	remaining: 26.9s
486:	learn: 0.2677036	total: 25.5s	remaining: 26.9s
487:	learn: 0.2677005	total: 25.5s	remaining: 26.8s
488:	learn: 0.2676778	total: 25.6s	remaining: 26.7s
489:	learn: 0.2676580	total: 25.6s	remaining: 26.7s
490:	learn: 0.2676535	total: 25.7s	remaining: 26.6s
491:	learn: 0.2676244	total: 25.7s	remaining: 26.6s
492:	learn: 0.2676156	total: 25.8s	remaining: 26.5s
493:	learn: 0.2676041	total: 25.8s	remaining: 26.4s
494:	learn: 0.2675953	total: 25.8s	remaining: 26.4s
495:	learn: 0.2675724	total: 25.9s	remaining: 26.3s
496:	learn: 0.2675140	total: 25.9s	remaining: 26.3s
497:	learn: 0.2675044	total: 26s	remaining: 26.2s
498:	learn: 0.2674786	total: 26s	remaining: 26.1s
499:	learn: 0.2674576	total: 26.1s	remaining: 26.1s
500:	learn: 0.267391

...

431:	learn: 0.2593573	total: 22.2s	remaining: 29.2s
432:	learn: 0.2593503	total: 22.3s	remaining: 29.1s
433:	learn: 0.2593477	total: 22.3s	remaining: 29.1s
434:	learn: 0.2593018	total: 22.4s	remaining: 29s
435:	learn: 0.2592742	total: 22.4s	remaining: 29s
436:	learn: 0.2592488	total: 22.5s	remaining: 28.9s
437:	learn: 0.2592259	total: 22.5s	remaining: 28.9s
438:	learn: 0.2591883	total: 22.6s	remaining: 28.8s
439:	learn: 0.2591882	total: 22.6s	remaining: 28.8s
440:	learn: 0.2591835	total: 22.6s	remaining: 28.7s
441:	learn: 0.2591251	total: 22.7s	remaining: 28.7s
442:	learn: 0.2590648	total: 22.8s	remaining: 28.6s
443:	learn: 0.2590505	total: 22.8s	remaining: 28.6s
444:	learn: 0.2590280	total: 22.9s	remaining: 28.5s
445:	learn: 0.2590231	total: 22.9s	remaining: 28.5s
446:	learn: 0.2590019	total: 23s	remaining: 28.4s
447:	learn: 0.2589937	total: 23s	remaining: 28.4s
448:	learn: 0.2589817	total: 23.1s	remaining: 28.3s
449:	learn: 0.2589786	total: 23.1s	remaining: 28.2s
450:	learn: 0.258952

590:	learn: 0.2540973	total: 31.4s	remaining: 21.7s
591:	learn: 0.2540714	total: 31.5s	remaining: 21.7s
592:	learn: 0.2540508	total: 31.5s	remaining: 21.6s
593:	learn: 0.2540327	total: 31.6s	remaining: 21.6s
594:	learn: 0.2539736	total: 31.6s	remaining: 21.5s
595:	learn: 0.2539684	total: 31.7s	remaining: 21.5s
596:	learn: 0.2539564	total: 31.8s	remaining: 21.4s
597:	learn: 0.2539222	total: 31.8s	remaining: 21.4s
598:	learn: 0.2538451	total: 31.9s	remaining: 21.3s
599:	learn: 0.2538275	total: 31.9s	remaining: 21.3s
600:	learn: 0.2537715	total: 32s	remaining: 21.3s
601:	learn: 0.2537389	total: 32.1s	remaining: 21.2s
602:	learn: 0.2537103	total: 32.1s	remaining: 21.2s
603:	learn: 0.2536875	total: 32.2s	remaining: 21.1s
604:	learn: 0.2536834	total: 32.3s	remaining: 21.1s
605:	learn: 0.2536622	total: 32.3s	remaining: 21s
606:	learn: 0.2536582	total: 32.4s	remaining: 21s
607:	learn: 0.2536386	total: 32.4s	remaining: 20.9s
608:	learn: 0.2536115	total: 32.5s	remaining: 20.9s
609:	learn: 0.2535

749:	learn: 0.2502698	total: 40.1s	remaining: 13.4s
750:	learn: 0.2502286	total: 40.2s	remaining: 13.3s
751:	learn: 0.2501938	total: 40.2s	remaining: 13.3s
752:	learn: 0.2501804	total: 40.3s	remaining: 13.2s
753:	learn: 0.2501670	total: 40.4s	remaining: 13.2s
754:	learn: 0.2501562	total: 40.4s	remaining: 13.1s
755:	learn: 0.2501182	total: 40.5s	remaining: 13.1s
756:	learn: 0.2500934	total: 40.5s	remaining: 13s
757:	learn: 0.2500626	total: 40.6s	remaining: 12.9s
758:	learn: 0.2500094	total: 40.6s	remaining: 12.9s
759:	learn: 0.2499681	total: 40.7s	remaining: 12.8s
760:	learn: 0.2499186	total: 40.7s	remaining: 12.8s
761:	learn: 0.2499048	total: 40.8s	remaining: 12.7s
762:	learn: 0.2498927	total: 40.8s	remaining: 12.7s
763:	learn: 0.2498671	total: 40.9s	remaining: 12.6s
764:	learn: 0.2498187	total: 40.9s	remaining: 12.6s
765:	learn: 0.2497935	total: 41s	remaining: 12.5s
766:	learn: 0.2497485	total: 41s	remaining: 12.5s
767:	learn: 0.2497112	total: 41.1s	remaining: 12.4s
768:	learn: 0.2496

908:	learn: 0.2468022	total: 50.1s	remaining: 5.02s
909:	learn: 0.2467878	total: 50.2s	remaining: 4.96s
910:	learn: 0.2467830	total: 50.2s	remaining: 4.91s
911:	learn: 0.2467297	total: 50.3s	remaining: 4.85s
912:	learn: 0.2466890	total: 50.3s	remaining: 4.8s
913:	learn: 0.2466792	total: 50.4s	remaining: 4.74s
914:	learn: 0.2466524	total: 50.4s	remaining: 4.69s
915:	learn: 0.2466472	total: 50.5s	remaining: 4.63s
916:	learn: 0.2466291	total: 50.6s	remaining: 4.58s
917:	learn: 0.2466063	total: 50.6s	remaining: 4.52s
918:	learn: 0.2465773	total: 50.7s	remaining: 4.47s
919:	learn: 0.2465665	total: 50.7s	remaining: 4.41s
920:	learn: 0.2465525	total: 50.8s	remaining: 4.36s
921:	learn: 0.2465058	total: 50.8s	remaining: 4.3s
922:	learn: 0.2465025	total: 50.9s	remaining: 4.25s
923:	learn: 0.2464996	total: 51s	remaining: 4.19s
924:	learn: 0.2464933	total: 51s	remaining: 4.13s
925:	learn: 0.2464900	total: 51.1s	remaining: 4.08s
926:	learn: 0.2464806	total: 51.1s	remaining: 4.02s
927:	learn: 0.2464

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=<catboost.core.CatBoostClassifier object at 0x0000000030DD5F98>,
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.035658, 0.04, 0.05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [49]:
best_cb = cb_model.best_estimator_
pred_proba_cb = best_cb.predict_proba(X_test)
roc_score = roc_auc_score(y_test, pred_proba_cb[:, 1])
print('roc на test', roc_score)

roc на test 0.933174653016176


In [50]:
SUBMIT_NUM = 12

pred_proba = best_cb.predict_proba(X_pred)[:, 1]
rep_df = rep.create_report(X_pred.index, pred_proba)
assert rep_df.user_id.nunique() == X_pred.index.nunique()
print ('Прогноз сохранен в файл ', rep.save_report(rep_df, SUBMIT_NUM))

print ('Распределение "вероятностей" модели')
pd.cut(pred_proba, 10).value_counts()

Прогноз сохранен в файл  predict_2019-05-23_submit_12.csv
Распределение "вероятностей" модели


(-5.81e-05, 0.101]    3515
(0.101, 0.201]         606
(0.201, 0.301]         706
(0.301, 0.401]         218
(0.401, 0.5]           209
(0.5, 0.6]             210
(0.6, 0.7]              90
(0.7, 0.8]              65
(0.8, 0.9]             104
(0.9, 1.0]             461
dtype: int64

In [ ]:
SUBMIT_NUM = 11

pred_proba = best_rf.predict_proba(X_pred)[:, 1]*0.1+best_xgb.predict_proba(X_pred)[:, 1]*0.9
rep_df = rep.create_report(X_pred.index, pred_proba)
assert rep_df.user_id.nunique() == X_pred.index.nunique()
print ('Прогноз сохранен в файл ', rep.save_report(rep_df, SUBMIT_NUM))

print ('Распределение "вероятностей" модели')
pd.cut(pred_proba, 10).value_counts()

In [61]:
X_train_scaled = StandardScaler().fit_transform(X_train)
X_test_scaled = StandardScaler().fit_transform(X_test)
X_pred_scaled = StandardScaler().fit_transform(X_pred)
# значение к метрике на кроссвалидации коррелирует к метрике на степике
rfcv = RandomForestClassifier(**best_rf.get_params())
cv_scores = cross_val_score(rfcv, X_train, y_train, scoring='roc_auc', cv=10, n_jobs=-1)
mean_cv_scores = np.mean(cv_scores)
print ('mean score', mean_cv_scores)

D:\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
D:\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
D:\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardS